### Add constraints of tuning parameters in the objective function

In this case, I have to construct several "measurements", build a model for each of them, then construct a matrix of coefficients where each row contains coefficients of each measurement.

Such an objective function is not linear! What to expect is hard to tell.

In [14]:
import poly_func as pfc

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import math
from scipy.optimize import minimize
from scipy.optimize import leastsq

In [69]:
def get_model(coefficient, n_paras, order):
    pfacts = pfc.all_facts(n_paras, order)
    
    def model(dp):
        return np.matmul(pfc.multiply(dp, pfacts), coefficient.transpose())
    
    return model

In [3]:
params = ['x1', 'x2']
order = 2
nscans = 6
k1 = 0.1
k2 = 5
k11 = 0.01
k22 = 0.05
k12 = 0
norm_val = 10
xmin = 1
xmax = 2

In [4]:
def generate_quad_data(nscans, p_err=0.01):
    dx1 = np.random.uniform(xmin, xmax, nscans)
    dx2 = np.random.uniform(xmin, xmax, nscans)
    norm = np.array([norm_val]* nscans)
    
    data = norm + k1*dx1 + k2*dx2 + k11*dx1*dx1 + k22*dx2*dx2 + k12*dx1*dx2
    error = data * np.random.normal(loc=0, scale=0.1, size=nscans)
    input_matrix = np.array([dx1, dx2])
    
    return (input_matrix, data+error, error)

In [5]:
generate_data = generate_quad_data

In [72]:
nscans = 30
in_matrix, data, error = generate_data(nscans, 0.05)
test_matrix, test_data, test_error = generate_data(1, 0.05)

In [73]:
P = pfc.getMatrixP(in_matrix, order)

In [74]:
objective = pfc.chi2_objective(P, data, error)

In [75]:
def get_N_coeffienct(n_x, order):
    N = 1
    for io in range(1, order+1):
        multi = 1
        for j in range(io):
            multi *= n_x + j
        multi /= math.factorial(io)
        N+= multi
    return N

In [76]:
X0 = [1]*int(get_N_coeffienct(len(params), order))

In [77]:
res = minimize(
    objective, X0, method='nelder-mead', 
    options={'xtol':1e-8, 'disp':True}
)
print(res.success)

False


In [78]:
X = res.x
print(X)

[ 12.17177316  -0.85849433   3.10154384  -0.23778283   1.08817597
   0.1468215 ]


In [79]:
print(in_matrix)
print(in_matrix.shape)

[[ 1.59211779  1.84133158  1.00174206  1.03079292  1.88981071  1.62393121
   1.60887976  1.14705054  1.88619143  1.21346997  1.3961274   1.36711437
   1.92351495  1.37663785  1.98753082  1.74395247  1.74668264  1.97443722
   1.22525189  1.33614329  1.72177446  1.8601663   1.76691486  1.64066533
   1.91308773  1.93548491  1.03396464  1.26305793  1.76900315  1.61171942]
 [ 1.34335339  1.97118453  1.77313755  1.81632796  1.04443536  1.33802556
   1.51434449  1.77545523  1.89992018  1.26762312  1.43745753  1.24637809
   1.78643404  1.89052254  1.74684314  1.15763409  1.02237112  1.10770829
   1.89891281  1.19905061  1.55987195  1.58193416  1.39017521  1.63386044
   1.72738874  1.65113316  1.01541164  1.08996491  1.17036189  1.32309198]]
(2, 30)


In [80]:
dp = in_matrix[:, 0]

In [81]:
model = get_model(X, len(params), order)
model(dp)

16.960996659063802

In [82]:
print(test_data)
print(test_matrix)

[ 13.29012097]
[[ 1.74037916]
 [ 1.01974254]]


In [83]:
model(test_matrix[:, 0])

15.204122581347649

In [84]:
def get_objective(coefficient, data, error, n_paras, order):
    pfacts = pfc.all_facts(n_paras, order)
    
    def model(dp):
        predi = np.matmul(pfc.multiply(dp, pfacts), coefficient.transpose())
        return sum( ((predi - data)/error)**2 )
    
    return model

In [85]:
obj_tune = get_objective(X, test_data, test_error, len(params), order)

In [86]:
P0 = [0.]*len(params)

In [87]:
res_tune = minimize(
    obj_tune, P0, method='nelder-mead', 
    options={'xtol':1e-8, 'disp':True}
)
print(res_tune.success)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 98
         Function evaluations: 183
True


In [88]:
obj_tune(res_tune.x)

0.0

In [89]:
test_data.shape

(1,)

In [90]:
model = get_model(X, len(params), order)
model(res_tune.x)

13.290120972090829

In [91]:
test_data

array([ 13.29012097])

In [92]:
res_tune.x

array([-0.36132796,  0.30481692])

In [93]:
test_matrix

array([[ 1.74037916],
       [ 1.01974254]])